In [ ]:
import keras as k

In [ ]:
# we want our coefficent high and our loss low!
# in general ML: penalty on high loss, so only high coefficients should get through and get higher accuracy

In [ ]:
smooth = 0.00001 # add a super small number to ensure we won't run into a division by 0 (aka the contribution it'll make is virtually nonexistent)

Dice Loss test (Dice coefficient as evaluation metric)

In [ ]:
def Dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def Dice_loss(y_true, y_pred):
    return 1.-dice_coef(y_true, y_pred)

# call the function
model.compile(optimizer=Adam(lr=1e-2), loss=Dice_loss, metrics=[Dice_coef])

BCE Dice Loss test (Dice coefficient as evaluation metric)

In [ ]:
from keras.losses import binary_crossentropy

# coefficient for here is the Dice coefficient too

def BCE_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + Dice_loss(y_true, y_pred)

# call the function
model.compile(optimizer = Adam(lr = 1e-2), loss=BCE_loss, metrics=[Dice_coef])

Jaccard Loss test (Jaccard coefficient as evaluation metric)

In [ ]:
def Jaccard_coef(y_true, y_predicted): # true and predicted binary mask (function that maps bit image into 0 and 1 pixel values)
    intersection = k.sum(y_true*y_predicted, axis=[2, -1, -2]) # k.sum for element wise multiplication, over what axis will the intersection be mapped
    sum_union = k.sum(y_true+y_predicted, axis=[2, -1, -2]) # union so addition
    JC = (intersection+smooth)/(sum_union-intersection+smooth) # formula for Jaccard coeff, but added smooth to prevent division by a zero!
    return k.mean(JC) # return mean of Jaccard coeffs (mean for the whole image dataset, 50 pics? 50 JC coeffs -> take mean to get 1)

def Jaccard_loss(y_true, y_predicted):
    return 1-Jaccard_coef(y_true, y_predicted) # make sure not JC!! bc that's inside the other func

# call the function
model.compile(optimizer=Adam(lr=1e-2), loss=Jaccard_loss, metrics=[Jaccard_coef]) # changed from 4 to 2

Focal Loss test (Dice coefficient as evaluation metric)
- address imbalances when doing image detection
- applies modulating term
- use gamma distribution to dynamically scale entropy loss (scaling factor is gamma)
- we can use dice coefficient
- focal loss is a generaliation of BCE/binary cross entropy which is used in dice loss, where you account for misclassified errors

In [ ]:
#Focal Loss  (pt) = -αt(1-  pt)^γ log(pt) #here's the equation

# uses the Dice coefficient instead of its own as well

def Focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):
    y_pred = K.clip(y_pred, K.epsilon(), 1.0 - K.epsilon())
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred)) # pt is probability
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

# call the function
model.compile(optimizer=Adam(lr=1e-2), loss=Focal_loss, metrics=[Dice_coef]) 